In [ ]:
import numpy as np
import cv2
import math
cap=cv2.VideoCapture(0)
whT=320
classFile='coco.txt'
classNames=[]

with open(classFile,'rt') as f:
    classNames=f.read().rstrip('\n').split('\n')
modelConfiguration='yolov3.cfg'
modelWeights='yolov3.weights'
net=cv2.dnn.readNetFromDarknet(modelConfiguration,modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

def findObject(outputs,img):
    hT,wT,cT=img.shape
    listt=[]
    bbox=[]
    classIds=[]
    confs=[]
    for output in outputs:
        for det in output:
            scores=det[5:]
            classId=np.argmax(scores)
            confidence=scores[classId]
            if confidence > 0.5:
                w,h=int(det[2]*wT),int(det[3]*hT)
                x,y=int((det[0]*wT)-w/2),int((det[1]*hT)-h/2)
                bbox.append([x,y,w,h])
                
                classIds.append(classId)
                confs.append(float(confidence))
    indices=cv2.dnn.NMSBoxes(bbox,confs,0.5,0.3)
    for i in indices:
        i=i[0]
        box=bbox[i]
        x,y,w,h=box[0],box[1],box[2],box[3]
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,255),2)
        centerx =x+w/2
        centery=y+h/2 
        area=h*w
        dist=(classNames[classIds[i]],[centerx,centery],area)
        if(dist[0]!=""):
            listt.append(dist)
        cv2.putText(img,f'{classNames[classIds[i]].upper()}{int(confs[i]*100)}%',(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.6
                   ,(255,0,255),2)
    diffx=0
    diffy=0
    line=""
    areas=""
    distance=0
    if len(listt)==2:
        print(listt)
        for objects in listt:
            diffx=objects[1][0]-diffx
            diffy=objects[1][1]-diffy
            line=line+" "+objects[0]
            areas=str(areas)+" "+str(objects[2])
        distance=math.sqrt(diffx**2+diffy**2)
    if line!=" " and distance!=0 and areas!=" ":
        print("Areas are"+ str(areas))
        print("distance of "+line)
        print(distance)
                   
while True:
    success,image=cap.read()
    blob=cv2.dnn.blobFromImage(image,1/255,(whT,whT),[0,0,0],1,crop=False)
    net.setInput(blob)
    layerNames=net.getLayerNames()
    outputNames=[layerNames[i[0]-1] for i in net.getUnconnectedOutLayers()]
    outputs=net.forward(outputNames)
    findObject(outputs,image)
#     print(listt) 
    net.getUnconnectedOutLayers()
    cv2.imshow('image',image)
    
    cv2.waitKey(1)
  

[('person', [290.0, 305.0], 158592), ('remote', [179.0, 340.5], 39150)]
Areas are 158592 39150
distance of  person remote
116.53862020806665
[('person', [290.5, 303.5], 139347), ('cell phone', [197.0, 329.5], 36144)]
Areas are 139347 36144
distance of  person cell phone
97.04766869945924
[('person', [289.5, 303.0], 136670), ('cell phone', [210.0, 342.0], 34776)]
Areas are 136670 34776
distance of  person cell phone
88.55083285887265
[('person', [290.5, 301.0], 136998), ('cell phone', [213.5, 345.0], 35178)]
Areas are 136998 35178
distance of  person cell phone
88.68483523128404
[('cell phone', [207.5, 343.5], 38367), ('person', [296.0, 301.5], 134292)]
Areas are 38367 134292
distance of  cell phone person
97.96045120353418
